In [ ]:
import os
import torch
import torchvision.transforms as transforms
import open_clip
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import torchvision.transforms as T
def process_video(video_name, label_file_path, output_dir, model, preprocess, root_dir, challenge, device):
    try:
        output_file = os.path.join(output_dir, f"{video_name}.txt")
        
        # Skip processing if output file already exists
        if os.path.exists(output_file):
            print(f"Skipping {video_name}: output file already exists.")
            return
        
        # Locate the video folder
        video_folder = None
        for batch in ['batch1', 'batch2']:
            possible_path = os.path.join(root_dir, batch, 'cropped_aligned', video_name)
            if os.path.exists(possible_path):
                video_folder = possible_path
                break

        if video_folder is None:
            print(f"Skipping {video_name}: video folder not found.")
            return

        # Read label file
        with open(label_file_path, 'r') as file:
            lines = file.readlines()[1:]  # Skip header row

        # Get sorted frame paths
        frame_paths = sorted([os.path.join(video_folder, frame) for frame in os.listdir(video_folder) if frame.endswith('.jpg')])

        with open(output_file, "w") as f:
            for frame_path, label in zip(frame_paths, lines):
                frame_label = list(map(float, label.strip().split(',')))

                # Load and preprocess the image
                image = Image.open(frame_path).convert('RGB')
                image = preprocess(image).unsqueeze(0).to(device)  # Prepare for model

                # Extract features
                with torch.no_grad():
                    features = model.encode_image(image).squeeze().cpu().numpy()

                # Write features and labels
                line = ','.join(map(str, features)) + ',' + ','.join(map(str, frame_label)) + '\n'
                f.write(line)

        print(f"Processed {video_name} -> {output_file}")

    except Exception as e:
        print(f"Error processing {video_name}: {str(e)}")

def extract_and_save_features(root_dir, label_dir, challenge, output_root):
    # Load CLIP model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-16", pretrained="openai")
    print(device)
    model.to(device)
    model.eval()

    challenge_folders = {'AU': 'AU_Detection_Challenge', 'EXPR': 'EXPR_Recognition_Challenge', 'VA': 'VA_Estimation_Challenge'}
    if challenge not in challenge_folders:
        raise ValueError(f"Invalid challenge: {challenge}. Must be 'AU', 'EXPR', or 'VA'.")

    challenge_folder = challenge_folders[challenge]
    train_label_folder = os.path.join(label_dir, challenge_folder, 'Train_Set')
    val_label_folder = os.path.join(label_dir, challenge_folder, 'Validation_Set')
    train_output_dir = os.path.join(output_root, challenge, 'training_set_features')
    val_output_dir = os.path.join(output_root, challenge, 'validation_set_features')
    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    label_files = [(folder, output_dir, f) for folder, output_dir in [(train_label_folder, train_output_dir), (val_label_folder, val_output_dir)] if os.path.exists(folder) for f in os.listdir(folder) if f.endswith('.txt')]
    
    if not label_files:
        raise FileNotFoundError(f"No label files found in {train_label_folder} or {val_label_folder}.")

    with ThreadPoolExecutor() as executor:
        futures = []
        with tqdm(total=len(label_files), desc="Processing videos") as pbar:
            for label_folder, output_dir, label_file in label_files:
                video_name = label_file.replace('.txt', '')
                label_file_path = os.path.join(label_folder, label_file)
                futures.append(executor.submit(process_video, video_name, label_file_path, output_dir, model, preprocess, root_dir, challenge, device))
            for future in futures:
                future.result()

    print(f"Feature extraction completed for {challenge}. Files saved in {output_root}/{challenge}")


In [2]:
# Define paths
root_dir = "../8th_ABAW"  # Replace with actual path (batch1, batch2)
label_dir = "6th_ABAW_Annotations"  # Replace with actual path

# Define transformations (resize, normalize, convert to tensor)
transform = T.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])]) 

In [ ]:
challenges = ["VA","EXPR","AU"]    
for i in challenges:
    extract_and_save_features(
  
        root_dir=root_dir,  # Root dataset directory
        label_dir=label_dir,  # Label directory
        challenge=i,  # 'AU', 'EXPR', or 'VA'
        output_root="Features_CLIP"  # Where to save extracted features
    )
